In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
workDir_path = os.getcwd() # Абс путь текущего рабочего каталога
trainDir_path = os.path.join(workDir_path, 'train') # Абс путь в папку с TrainData
testDir_path = os.path.join(workDir_path, 'test') # Абс путь в папку с TestData

parquetsTrain = os.listdir(path = trainDir_path) # Список "паркетов"
parquetsTest = os.listdir(path = testDir_path) # Список "паркетов"

In [3]:
# категор фичи
cat_columns = ['currency', 'operation_kind', 'operation_type_group', 'ecommerce_flag', 'payment_system',
               'income_flag', 'country', 'mcc_category', 'city', 'card_type', 'mcc', 'operation_type']

# числовые фичи
num_columns = ['amnt', 'hour','hour_diff', 'days_before', 'day_of_week']

# фичи с которыми работаем
columns = ['app_id'] + cat_columns + num_columns


In [104]:
# Отладка func
# path = os.path.join(trainDir_path, parquetsTrain[0]) # Абс путь до "паркета"
# X = pd.read_parquet(path)
# a=17.8209044625170474069 #Заточили под 100
# X['amnt'] = np.exp(X['amnt']*a)-1

In [139]:
def make_pivot_table_feature2(df, names, aggfuncs=[], normalize=False):
    """
    ....
    """
    tmp = pd.pivot_table(df, values='amnt', index='app_id', columns=names, aggfunc=['sum', 'mean'] + aggfuncs )
#     names = pd.Series(names).apply('sum')
    tmp.columns = [str('_').join(names) + '__' + str('___').join([str(x)for x in i]) for i in tmp.columns]
    if normalize: # особо и не нужна 
        tmp = tmp.div(tmp.sum(axis=1), axis=0)
        tmp.fillna(0.0)
    tmp = tmp.reset_index()

    return (tmp.fillna(0))

In [140]:
def func(data, cat_columns=cat_columns, num_columns=num_columns, name='None'):
    '''
    Препроцессинг данных
    '''
    coef = 17.8209044625170474069
    data['amnt'] = np.exp(data['amnt']*coef)-1
    
    a = make_pivot_table_feature2(data, ['income_flag', 'mcc_category'], aggfuncs=['std'])
    b = make_pivot_table_feature2(data, ['income_flag', 'operation_kind'], aggfuncs=['std'])
    c = make_pivot_table_feature2(data, ['income_flag', 'day_of_week', 'operation_type_group'], aggfuncs=['std'])
    return a.merge(b).merge(c)

In [141]:
def read_data(folder_name, mode, output_name, columns=columns, func=func):
    '''
    folder_name - папка где хранятся обработанные файлы
    mode = train/test
    output_name - имя выходного файла
    columns - какие признаки исходного датасета читаем
    func - функция предобработки признаков
    return: сохраняет конечный файл в формате csv/parquet
    '''
    
    # Режим работы
    if mode == 'train':
        parquets = parquetsTrain
        dataDir_path = trainDir_path
    elif mode == 'test':
        parquets = parquetsTest
        dataDir_path = testDir_path
    
    # Проверка от создания уже существующей папки
    if os.path.isdir(folder_name) is True:
        raise BaseException('Папка уже существует, выбери другое имя')
    
    # Создаем каталог
    os.mkdir(folder_name)
    
    # Для каждого паркета запускаем обработчик   
    for parquet in parquets:
        path = os.path.join(dataDir_path, parquet) # Абс путь до "паркета"
        name = path.split('\\')[-1][:8] # получаем имя
        data = pd.read_parquet(path, columns=columns) 
        
        # Генерация фичей
        data = func(data, name=name)
        
        # Сохраняем данные
        data.to_parquet('./{0}/{1}.csv'.format(folder_name, name))        
        print('{0} обработан'.format(name))
    # Склейка датасета
    dir_path = os.path.join(workDir_path, folder_name) # Директория с предобработанными данными
    parquets = os.listdir(path = dir_path) # Список "паркетов"
    
    data = False # DATA
    p = 0
    for parquet in parquets:
        path = os.path.join(dir_path, parquet) # Абс путь до "паркета"
        if data is False: # Для первого "паркета"
            data = pd.read_parquet(path)
            continue
            
        _ = pd.read_parquet(path)
        data = pd.concat([data, _]) # Соединить таблички
        p+=1
        print(p)
    print('Сохраняю')
    data.to_parquet('{0}.parquet'.format(output_name)) # ДАННЫЕ СОХРАНЯЮТСЯ В ФОРМАТЕ parquet! а не csv! pd.read_parquet()!
    print('OK')

In [142]:
read_data(folder_name='test-26-12-2020', mode='test', output_name='test_data26122020_dopFeatures') # Обработка теста

part_000 обработан
part_001 обработан
part_002 обработан
part_003 обработан
part_004 обработан
part_005 обработан
part_006 обработан
part_007 обработан
part_008 обработан
part_009 обработан
part_010 обработан
part_011 обработан
part_012 обработан
part_013 обработан
part_014 обработан
part_015 обработан
part_016 обработан
part_017 обработан
part_018 обработан
part_019 обработан
part_020 обработан
part_021 обработан
part_022 обработан
part_023 обработан
part_024 обработан
part_025 обработан
part_026 обработан
part_027 обработан
part_028 обработан
part_029 обработан
part_030 обработан
part_031 обработан
part_032 обработан
part_033 обработан
part_034 обработан
part_035 обработан
part_036 обработан
part_037 обработан
part_038 обработан
part_039 обработан
part_040 обработан
part_041 обработан
part_042 обработан
part_043 обработан
part_044 обработан
part_045 обработан
part_046 обработан
part_047 обработан
part_048 обработан
part_049 обработан
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20

In [ ]:
# Потом запустить тест на обработку

In [143]:
read_data(folder_name='train-26-12-2020', mode='train', output_name='train_data26122020') # обработка трейна

part_000 обработан
part_001 обработан
part_002 обработан
part_003 обработан
part_004 обработан
part_005 обработан
part_006 обработан
part_007 обработан
part_008 обработан
part_009 обработан
part_010 обработан
part_011 обработан
part_012 обработан
part_013 обработан
part_014 обработан
part_015 обработан
part_016 обработан
part_017 обработан
part_018 обработан
part_019 обработан
part_020 обработан
part_021 обработан
part_022 обработан
part_023 обработан
part_024 обработан
part_025 обработан
part_026 обработан
part_027 обработан
part_028 обработан
part_029 обработан
part_030 обработан
part_031 обработан
part_032 обработан
part_033 обработан
part_034 обработан
part_035 обработан
part_036 обработан
part_037 обработан
part_038 обработан
part_039 обработан
part_040 обработан
part_041 обработан
part_042 обработан
part_043 обработан
part_044 обработан
part_045 обработан
part_046 обработан
part_047 обработан
part_048 обработан
part_049 обработан
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20